# AI Fall 2022 - A4 - Decision Tree
## Your info

**Student Name:** Mehran Mazaheri

**Student Id:** 98102346

## Practical Question - Decision Tree (25 + 5 points)
In this question you will first write a program to find an appropriate order of features for training a decision tree based on information gain. In the second part, you will use existing implementations to train and tune a classifier using GridSearch on the given dataset. And try to finish this in <15 mins.

### Import
Import the libraries you need in the next cell.

In [126]:
import pandas as pd
from math import log2

# ...

### Part A (20 points)
Consider the following dataframe and answer the following questions:

In [127]:
df = pd.DataFrame({
    'Race': [1.0, 2.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 2.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0],
    'Sex': [1.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 1.0, 1.0, 1.0, 2.0, 1.0, 2.0, 1.0, 1.0, 2.0, 1.0, 1.0, 2.0],
    'Age': [43.0, 72.0, 4.0, 59.0, 34.0, 55.0, 41.0, 39.0, 6.0, 77.0, 67.0, 35.0, 45.0, 39.0, 32.0, 57.0, 99.0, 68.0, 47.0, 28.0],
    'BMI': [25.803, 33.394, 20.278, 32.918, 30.743, 37.734, 23.725, 20.722, 19.452, 22.044, 17.481, 18.918, 28.578, 19.214, 23.562, 23.446, 21.872, 20.691, 25.471, 25.82],
    'Smoker': [1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0],
    'Label': [False, True, False, False, False, True, False, False, False, False, True, False, False, False, False, False, False, False, True, False],
})

df.head()

,Race,Sex,Age,BMI,Smoker,Label
0,1.0,1.0,43.0,25.803,1.0,False
1,2.0,2.0,72.0,33.394,0.0,True
2,1.0,2.0,4.0,20.278,1.0,False
3,1.0,2.0,59.0,32.918,0.0,False
4,1.0,2.0,34.0,30.743,1.0,False


#### QA.1 (10 points)
Complete the following function that calculates and returns the list of features based on their information gain.

In [128]:
def entropy(label: list) -> float:
    seen = []
    H = 0
    numberOfSamples = len(label)
    for value in label:
        if value not in seen:
            seen.append(value)
            numberOfOccurrence = label.count(value)
            if numberOfOccurrence < numberOfSamples:
                H -= numberOfOccurrence/numberOfSamples * log2(numberOfOccurrence/numberOfSamples)
    return H

def conditionalEntropy(feature: list, label: list) -> float:
    seen = []
    numberOfSamples = len(feature)
    H = 0
    for value in feature: 
        if value not in seen:
            seen.append(value)
            indices = [i for i, e in enumerate(feature) if e == value]
            numberOfValue = len(indices)
            newLabel = [label[i] for i in indices]
            H += (numberOfValue/numberOfSamples) * entropy(newLabel)
    return H

In [129]:
def feature_order(X_data:pd.DataFrame, y_data:pd.Series):
    '''
    Calculate the best feature order
    for training a decision tree based on
    information gain.
    Input:
        X_data (pd.DataFrame) contains data features
        y_data (pd.Series) contains the labels
    Output:
        order (list[str]): The `X_data` columns in the correct order
    '''
    informationGain = {}
    for col in X_data.columns:
        informationGain[col] = entropy(list(y_data)) - conditionalEntropy(list(df.get(col)), list(y_data))
    informationGain = dict(sorted(informationGain.items(), key=lambda item: -item[1]))
    print("Information Gains:\n", informationGain)

    order = list(informationGain.keys())
    return order


feature_order(df.get(df.columns[:-1]), df.Label)

Information Gains:
 {'Age': 0.7219280948873623, 'BMI': 0.7219280948873623, 'Race': 0.03690791540384364, 'Smoker': 0.03051882909598158, 'Sex': 0.0018149273570102764}


['Age', 'BMI', 'Race', 'Smoker', 'Sex']

#### QA.2 (5 points)
What are some of the effective approaches to prevent the tree from overfitting? (three approaches)

<font color=#183364>
    <i>
        Answer.
        <ol>
            <li></li>
            <li></li>
            <li></li>
        </ol>
    </i>
</font>

#### QA.3 (5 points)
About the `Age` feature which is an integer-valued input attribute, how can a tree utilize it for classification? Write a small piece of code to find the answer for this specific column.

<font color=#183364>
    <i>
        Answer.
    </i>
</font>

In [ ]:
# Your code here

### Part B (10 points)
In this part you will learn more about [sklearn.tree.DecisionTreeClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html) and use it to train a classifier and tune its parameters.

#### QB.1 (5 points)
For measuring the _accuracy_ of your models, choose a subset of the given dataset as your validation set (Check out [sklearn.model_selection.train_test_split](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)).

In [ ]:
# Your code here
## Split the dataset


In [ ]:
# Your code here
## Set your own hyperparameters below, such as:
dt_hyperparams = {
    'min_samples_split': 2,
    'criterion': 'entropy'
    # ...
}

## Train the model and check its performance


#### QB.2 (5 points)
Take a look at customizable and potentially useful input params of the model, define a range for each of their values, and use GridSearch (Feel free to implement/[import](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) it) and find a set of hyperparametetrs that work best among your candidates.

In [ ]:
# Your code here

#### QB.3 (5 extra points)
Visualize the tree splits and interpret it (Hint: Search about `IPython.display.Image`, `pydotplus`, `StringIO`, and `export_graphviz`).